# TDSP Stage 4 – Deployment & Business Insights

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada **interpretasi hasil modeling**, **penerjemahan segmentasi ke insight bisnis**,
serta **penyusunan output yang siap digunakan oleh stakeholder**.

## 4.1 Tujuan Deployment

Tujuan utama tahap deployment adalah:

1. Menyajikan hasil segmentasi pelanggan dalam bentuk yang mudah dipahami.
2. Mengaitkan setiap cluster dengan karakteristik bisnis yang jelas.
3. Menghasilkan insight yang dapat langsung ditindaklanjuti (*actionable*).
4. Menyiapkan artefak analitik yang siap diintegrasikan ke proses bisnis.


## 4.2 Ringkasan Segmentasi Pelanggan

Segmentasi pelanggan dihasilkan menggunakan **K-Means (k = 5)** berdasarkan fitur RFM.

Setiap cluster merepresentasikan **pola perilaku transaksi yang berbeda**, sehingga strategi bisnis
tidak dapat disamaratakan untuk seluruh pelanggan.

In [38]:
# ==============================================================
# 4.2 Re-initialize Spark & Load Data
# ==============================================================
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder
    .appName("RFM-Deployment")
    .getOrCreate()
)

# Gunakan path file yang benar-benar ada di sistem Anda
PATH_REAL = "../datasets/rfm_clustered_results.csv.gz"

# Load data dengan deteksi otomatis kompresi gzip
df_final = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(PATH_REAL)
)

print(f"Total Pelanggan Tersegmentasi: {df_final.count()}")
df_final.show(5)

Total Pelanggan Tersegmentasi: 3920
+----------+-------+---------+------------------+-------+
|CustomerID|Recency|Frequency|          Monetary|Cluster|
+----------+-------+---------+------------------+-------+
|   15039.0|      9|       42|19914.439999999966|      3|
|   17966.0|     37|        4|           1098.43|      0|
|   16553.0|    163|        6| 5719.819999999999|      2|
|   17955.0|    198|        3|             557.3|      2|
|   13178.0|     26|       11| 5725.470000000003|      4|
+----------+-------+---------+------------------+-------+
only showing top 5 rows



## 4.3 Profil RFM per Cluster

Untuk memahami karakteristik tiap segmen, dilakukan agregasi statistik RFM
pada level cluster.

Profil ini menjadi dasar utama dalam menentukan **strategi marketing**.

In [39]:
# ==============================================================
# 4.3 Agregasi Profil RFM (Spark Version)
# ==============================================================

cluster_stats = df_final.groupBy("Cluster").agg(
    F.count("CustomerID").alias("Total_Customers"),
    F.round(F.avg("Recency"), 1).alias("Avg_Recency"),
    F.round(F.avg("Frequency"), 1).alias("Avg_Frequency"),
    F.round(F.avg("Monetary"), 2).alias("Avg_Monetary")
).orderBy("Cluster")

cluster_stats.show()

+-------+---------------+-----------+-------------+------------+
|Cluster|Total_Customers|Avg_Recency|Avg_Frequency|Avg_Monetary|
+-------+---------------+-----------+-------------+------------+
|      0|           2603|       44.9|          3.0|     1114.35|
|      1|              3|        2.7|         18.3|    207560.2|
|      2|            962|      247.0|          1.5|       493.0|
|      3|             25|       17.8|         45.2|     45946.6|
|      4|            327|       14.3|         14.1|     6611.98|
+-------+---------------+-----------+-------------+------------+



## 4.4 Interpretasi Bisnis per Cluster

Berdasarkan profil RFM, cluster dapat diinterpretasikan secara konseptual sebagai berikut:

- **Cluster 0 – Low Value / Dormant Customers**  
  Recency tinggi, Frequency rendah, Monetary rendah. Pelanggan jarang bertransaksi dan berisiko churn.

- **Cluster 1 – Occasional Buyers**  
  Frequency rendah–menengah dengan nilai transaksi terbatas.

- **Cluster 2 – Regular Customers**  
  Pelanggan dengan pola belanja stabil dan kontribusi moderat.

- **Cluster 3 – High Value Customers**  
  Frequency dan Monetary tinggi, memberikan kontribusi signifikan terhadap revenue.

- **Cluster 4 – Very High Value / VIP Customers**  
  Pelanggan paling bernilai, jumlah kecil namun kontribusi revenue sangat besar.


In [40]:
# ==============================================================
# 4.4 Mapping Nama Segmen (Spark Logic)
# ==============================================================

# Mapping ini didasarkan pada logika umum RFM K-Means
df_segmented = df_final.withColumn("Segment_Name", 
    F.when(F.col("Cluster") == 4, "VIP / Loyalists")
     .when(F.col("Cluster") == 3, "High Value")
     .when(F.col("Cluster") == 2, "Regular Customers")
     .when(F.col("Cluster") == 1, "Occasional Buyers")
     .otherwise("Dormant / At Risk")
)

# Menghitung persentase distribusi segmen
total_count = df_segmented.count()
segment_dist = (df_segmented.groupBy("Segment_Name")
                .count()
                .withColumn("Percentage", F.round((F.col("count") / total_count) * 100, 2))
                .orderBy(F.desc("Percentage")))

segment_dist.show()

+-----------------+-----+----------+
|     Segment_Name|count|Percentage|
+-----------------+-----+----------+
|Dormant / At Risk| 2603|      66.4|
|Regular Customers|  962|     24.54|
|  VIP / Loyalists|  327|      8.34|
|       High Value|   25|      0.64|
|Occasional Buyers|    3|      0.08|
+-----------------+-----+----------+



## 4.5 Rekomendasi Strategi Bisnis

Berdasarkan segmentasi pelanggan, rekomendasi strategis yang dapat diterapkan:

### Cluster Low Value / Dormant
- Kampanye reaktivasi (voucher, reminder)
- Evaluasi biaya marketing (hindari overspending)

### Cluster Regular Customers
- Upselling dan cross-selling
- Program loyalitas ringan

### Cluster High & Very High Value
- Program VIP / loyalty eksklusif
- Personalisasi penawaran
- Prioritas layanan pelanggan

Pendekatan ini memungkinkan perusahaan memaksimalkan ROI dari aktivitas marketing.

In [41]:
# ==============================================================
# 4.5 Revenue Contribution Analysis
# ==============================================================

revenue_summary = df_segmented.groupBy("Segment_Name").agg(
    F.sum("Monetary").alias("Total_Revenue")
).withColumn("Revenue_Share_%", 
    F.round((F.col("Total_Revenue") / df_segmented.select(F.sum("Monetary")).collect()[0][0]) * 100, 2)
).orderBy(F.desc("Total_Revenue"))

revenue_summary.show()

+-----------------+------------------+---------------+
|     Segment_Name|     Total_Revenue|Revenue_Share_%|
+-----------------+------------------+---------------+
|Dormant / At Risk|       2900659.383|          39.69|
|  VIP / Loyalists| 2162118.180000002|          29.58|
|       High Value|1148665.0500000003|          15.72|
|Occasional Buyers| 622680.5900000001|           8.52|
|Regular Customers| 474268.3509999996|           6.49|
+-----------------+------------------+---------------+



## 4.6 Artefak Deployment

Output dari analisis ini dapat dideploy dalam berbagai bentuk:

- Tabel segmentasi pelanggan (PostCode → Cluster)
- Dashboard BI (Power BI / Tableau / Fabric)
- Dataset input untuk kampanye marketing
- Dasar untuk analisis lanjutan (CLV, churn prediction)


In [42]:
import os
import sys

# Sesuaikan dengan lokasi folder hadoop Anda
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["hadoop.home.dir"] = r"C:\hadoop"
os.environ["PATH"] += r";C:\hadoop\bin"

# Tambahkan konfigurasi agar Spark tidak mencoba mengecek permission ala Linux di Windows
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("RFM-Deployment-Final")
    .config("spark.hadoop.fs.permissions.umask-mode", "000")
    .getOrCreate()
)

In [43]:
# ==============================================================
# 4.6 Export Artefak Final (Pandas Bridge - GZ Version)
# ==============================================================

# Gunakan ekstensi .csv.gz
OUTPUT_FILE_GZ = "../datasets/online_retail_customer_segments_final.csv.gz"

# 1. Konversi Spark DataFrame ke Pandas
# Kita ambil kolom yang relevan untuk kebutuhan bisnis/marketing
df_final_pandas = df_segmented.select(
    "CustomerID", 
    "Segment_Name", 
    "Recency", 
    "Frequency", 
    "Monetary", 
    "Cluster"
).toPandas()

# 2. Simpan sebagai CSV tunggal dengan kompresi GZIP
df_final_pandas.to_csv(
    OUTPUT_FILE_GZ, 
    index=False, 
    compression='gzip'
)

print(f"✅ Artefak deployment berhasil disimpan: {OUTPUT_FILE_GZ}")

✅ Artefak deployment berhasil disimpan: ../datasets/online_retail_customer_segments_final.csv.gz


## Ringkasan Stage 4

Pada TDSP Stage 4 ini, kita telah:
- Menginterpretasikan hasil clustering pelanggan
- Mengaitkan cluster dengan konteks dan strategi bisnis
- Menyiapkan artefak analitik untuk deployment

Tahap terakhir adalah **TDSP Stage 5 – Customer Acceptance**,
yang berfokus pada validasi hasil bersama stakeholder dan perencanaan tindak lanjut.